In [101]:
import sys
sys.path.append('/opt/code/face/dual_shot')
sys.path.append('/opt/code/face/dual_shot/prepare_data')
import numpy as np
import tensorflow as tf
from dual_conf import current_config as conf
from prepare_data.generator import gen_data
from prepare_data.model_target import cal_target
from prepare_data.generator import image_reader,layer_strides,map_size,e_scale,ratio
from prepare_data.model_target import init_anchors
from dual_conf import current_config as conf
from prepare_data.widerface import WIDERFaceDetection

config = tf.ConfigProto()
config.gpu_options.allow_growth= True
sess = tf.Session(config=config)

## load gt

In [2]:
test_set = WIDERFaceDetection(conf.data_path, 'train', None, None)
image, gts, labels = image_reader(test_set)

Error bbox number to filter : 48,  bbox number: 159376


## initialize anchors

In [3]:
anchors = init_anchors(layer_strides, map_size, ratio, e_scale)
anchors.shape

(34125, 4)

## calculate target

In [177]:
gen = gen_data(conf.batch_size, 'train')
print([i.shape for i in next(gen)[0]])
[batch_img, e_reg_targets, e_cls_targets, o_reg_targets, o_cls_targets], []= next(gen)

Error bbox number to filter : 48,  bbox number: 159376
[(4, 640, 640, 3), (4, 34125, 4), (4, 34125), (4, 34125, 4), (4, 34125)]


## cls loss demo

In [185]:
cls_target = tf.Variable(e_cls_targets)
shape = [4,conf.num_anchor,2]
predict_logits= tf.random_uniform(shape,1,10)
print(cls_target,predict_logits)

<tf.Variable 'Variable_21:0' shape=(4, 34125) dtype=int64_ref> Tensor("random_uniform_26:0", shape=(4, 34125, 2), dtype=float32)


In [186]:
train_indices = tf.where(tf.not_equal(cls_target, 0))  # 参与训练的indice
train_indices

<tf.Tensor 'Where_32:0' shape=(?, 2) dtype=int64>

In [179]:
cls_target = tf.gather_nd(cls_target, train_indices)
cls_target = tf.where(cls_target > 0, cls_target, tf.zeros_like(cls_target))  # change negative tag from -1 to 0
cls_target = tf.cast(cls_target, dtype=tf.int32)
cls_target = tf.one_hot(cls_target, depth=2)

In [180]:
predict_logits[..., 0].shape

TensorShape([Dimension(4), Dimension(34125)])

In [181]:
# remove padding anchors and un-trained anchors from pred logit
logit0 = tf.gather_nd(predict_logits[..., 0], train_indices)
logit1 = tf.gather_nd(predict_logits[..., 1], train_indices)
logit = tf.stack([logit0, logit1], axis=1)

loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=predict_logits, logits=logit)
loss

## cls loss func

In [182]:
import keras.backend as K
def cls_loss(cls_target, predict_logits):
    """
    :param cls_target:2-d array,[batch_num,num_train_anchors]; 1,-1,0 for pos,neg and un-train anchors respectively
    :param predict_logits: 2-d array,(batch_num,anchors_num,2) fg or bg
    :return: classification loss of training anchors
    """

    # remove padding from true class
    train_indices = tf.where(tf.not_equal(cls_target, 0))  # 参与训练的indice
    cls_target = tf.gather_nd(cls_target, train_indices)
    cls_target = tf.where(cls_target > 0, cls_target, tf.zeros_like(cls_target))  # change negative tag from -1 to 0
    cls_target = tf.cast(cls_target, dtype=tf.int32)
    cls_target = tf.one_hot(cls_target, depth=2)

    # remove padding anchors and un-trained anchors from pred logit
    logit0 = tf.gather_nd(predict_logits[..., 0], train_indices)
    logit1 = tf.gather_nd(predict_logits[..., 1], train_indices)
    logit = tf.stack([logit0, logit1], axis=1)

    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=cls_target, logits=logit)
    loss = K.mean(loss)
    return loss

In [183]:
cls_target = tf.Variable(e_cls_targets)
shape = [4,conf.num_anchor,2]
predict_logits= tf.random_uniform(shape,1,10)
print(cls_target,predict_logits)

<tf.Tensor 'Mean_6:0' shape=() dtype=float32>

In [184]:
to_run = [cls_loss(cls_target, predict_logits)]
sess.run(tf.global_variables_initializer())
for i in to_run:
    print(i,'\n',sess.run(i))

Tensor("Mean_7:0", shape=(), dtype=float32) 
 0.71102315


## regr loss func

In [172]:
def regr_loss(reg_target, predict_deltas, cls_target):
    """
    :param reg_target: 2-d array,[batch_num,num_anchors,(dy,dx,dz,dh)]fg or bg
    :param predict_deltas: 2-d array,[batch_num,num_anchors,(dy,dx,dz,dh)] fg or bg
    :param cls_target: 2-d array,[batch_num,num_anchors]; 1,-1,0 for pos,neg and un-train anchors respectively
    :return:
    """
    # remove un-trained anchors
    pos_indice = tf.where(tf.equal(cls_target, 1))
    reg_target = tf.gather_nd(reg_target, pos_indice)
    pred_deltas = tf.gather_nd(predict_deltas, pos_indice)
    
    reg_target = tf.cast(reg_target,tf.float64)
    pred_deltas = tf.cast(pred_deltas,tf.float64)
    diff = tf.abs(reg_target - pred_deltas)
    smooth_loss = tf.where(diff < 1, tf.pow(diff, 2) * 0.5, diff - 0.5)
    loss = K.mean(smooth_loss)
    return loss

In [173]:
reg_target = tf.Variable(e_reg_targets)
cls_target = tf.Variable(e_cls_targets)
shape = [4,conf.num_anchor,4]
predict_deltas= tf.random_uniform(shape)
print(reg_target,cls_target,predict_deltas)

<tf.Variable 'Variable_18:0' shape=(4, 34125, 4) dtype=float64_ref> <tf.Variable 'Variable_19:0' shape=(4, 34125) dtype=int64_ref> Tensor("random_uniform_24:0", shape=(4, 34125, 4), dtype=float32)


In [174]:
to_run = [regr_loss(reg_target, predict_deltas, cls_target)]
sess.run(tf.global_variables_initializer())
for i in to_run:
    print(i,'\n',sess.run(i))

Tensor("Mean_5:0", shape=(), dtype=float64) 
 0.9879738810606629
